<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_11_24_Directly_Prompt_LLM_via_Conversation_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to directly prompt an LLM, bypassing RAG content retrieval.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [7]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [8]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [9]:
from typing import List, Optional

async def create_openai_specification(model: enums.OpenAIModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"OpenAI [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=model,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def prompt(prompt: str, specification_id: str, messages: Optional[List[input_types.ConversationMessageInput]] = None):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.prompt(prompt=prompt, specification=input_types.EntityReferenceInput(id=specification_id), messages=messages)

        return response.prompt.messages if response.prompt is not None and response.prompt.messages is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)


Execute Graphlit example

In [10]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing specifications; only needed for notebook example
await delete_all_specifications()

print('Deleted all specifications.')


Deleted all specifications.


Create OpenAI GPT-4o specification.

In [11]:
messages = None

specification_id = await create_openai_specification(enums.OpenAIModels.GPT4O_128K)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    user_prompt = "Discuss the history of hockey."

    messages = await prompt(user_prompt, specification_id)

    if messages is not None:
        display(Markdown('### Conversation:'))

        for message in messages:
            if message is not None:
                display(Markdown(f'**{message.role}:**\n{message.message}'))

                if message.role == enums.ConversationRoleTypes.ASSISTANT:
                    print()

        print()

Created specification [733c3bc4-f863-4547-96d2-fee788325134].


### Conversation:

**USER:**
Discuss the history of hockey.

**ASSISTANT:**
Hockey is a sport with a rich and varied history, evolving over centuries into the modern game we know today. Its origins can be traced back to ancient civilizations, where games involving sticks and balls were played. Here’s a brief overview of the history of hockey:

### Ancient and Medieval Origins
- **Ancient Games**: Games resembling hockey have been played for thousands of years. Evidence suggests that ancient Egyptians, Greeks, and Romans played stick-and-ball games. In particular, a game called "hurling" was played in Ireland, and "shinty" in Scotland, both of which are considered precursors to modern hockey.
- **Medieval Europe**: During the Middle Ages, various stick-and-ball games were played across Europe. These games were often played on fields and involved teams trying to hit a ball into a goal.

### Development of Modern Hockey
- **Field Hockey**: The modern form of field hockey began to take shape in England in the 18th and 19th centuries. The first formal set of rules was established in the mid-19th century, and the first hockey club, Blackheath Hockey Club, was founded in 1861. The sport spread throughout the British Empire, gaining popularity in countries like India and Australia.
- **Ice Hockey**: Ice hockey is believed to have developed in Canada in the 19th century, influenced by various stick-and-ball games brought by European immigrants. The first recorded indoor ice hockey game took place in Montreal in 1875. The sport quickly gained popularity, leading to the formation of amateur clubs and leagues.

### Professionalization and Global Spread
- **National Hockey League (NHL)**: The NHL was founded in 1917 in Montreal, Canada, and has since become the premier professional ice hockey league in the world. It began with just a few teams and has expanded significantly over the years.
- **International Competitions**: Ice hockey became an Olympic sport in 1920, and the International Ice Hockey Federation (IIHF) was established to govern the sport globally. Field hockey was included in the Olympics in 1908 for men and 1980 for women.

### Modern Era
- **Field Hockey**: Today, field hockey is played worldwide, with significant followings in Europe, Asia, Africa, and Oceania. The Hockey World Cup and the Olympic Games are the sport's most prestigious tournaments.
- **Ice Hockey**: Ice hockey remains extremely popular in North America and Europe, with the NHL being the most prominent league. The sport is also played at various levels, from amateur to professional, and is a staple of the Winter Olympics.

### Cultural Impact
Hockey, both field and ice, has had a significant cultural impact, particularly in countries where it is a major sport. It has inspired countless players and fans and has become an integral part of national identities, especially in countries like Canada, where ice hockey is considered a national pastime.

Overall, hockey's history is a testament to its enduring appeal and adaptability, evolving from ancient games to a modern sport enjoyed by millions around the world.

In [12]:
        input_messages = [
            input_types.ConversationMessageInput(message=message.message, role=message.role)
            for message in messages if message is not None and message.message is not None
        ]

        user_prompt = "Great, but focus specifically on the history of hockey in Canada."

        messages = await prompt(user_prompt, specification_id, input_messages)

        if messages is not None:
            display(Markdown('### Conversation:'))

            for message in messages:
                if message is not None:
                    display(Markdown(f'**{message.role}:**\n{message.message}'))

                    if message.role == enums.ConversationRoleTypes.ASSISTANT:
                        print()

            print()

### Conversation:

**USER:**
Discuss the history of hockey.

**ASSISTANT:**
Hockey is a sport with a rich and varied history, evolving over centuries into the modern game we know today. Its origins can be traced back to ancient civilizations, where games involving sticks and balls were played. Here’s a brief overview of the history of hockey:

### Ancient and Medieval Origins
- **Ancient Games**: Games resembling hockey have been played for thousands of years. Evidence suggests that ancient Egyptians, Greeks, and Romans played stick-and-ball games. In particular, a game called "hurling" was played in Ireland, and "shinty" in Scotland, both of which are considered precursors to modern hockey.
- **Medieval Europe**: During the Middle Ages, various stick-and-ball games were played across Europe. These games were often played on fields and involved teams trying to hit a ball into a goal.

### Development of Modern Hockey
- **Field Hockey**: The modern form of field hockey began to take shape in England in the 18th and 19th centuries. The first formal set of rules was established in the mid-19th century, and the first hockey club, Blackheath Hockey Club, was founded in 1861. The sport spread throughout the British Empire, gaining popularity in countries like India and Australia.
- **Ice Hockey**: Ice hockey is believed to have developed in Canada in the 19th century, influenced by various stick-and-ball games brought by European immigrants. The first recorded indoor ice hockey game took place in Montreal in 1875. The sport quickly gained popularity, leading to the formation of amateur clubs and leagues.

### Professionalization and Global Spread
- **National Hockey League (NHL)**: The NHL was founded in 1917 in Montreal, Canada, and has since become the premier professional ice hockey league in the world. It began with just a few teams and has expanded significantly over the years.
- **International Competitions**: Ice hockey became an Olympic sport in 1920, and the International Ice Hockey Federation (IIHF) was established to govern the sport globally. Field hockey was included in the Olympics in 1908 for men and 1980 for women.

### Modern Era
- **Field Hockey**: Today, field hockey is played worldwide, with significant followings in Europe, Asia, Africa, and Oceania. The Hockey World Cup and the Olympic Games are the sport's most prestigious tournaments.
- **Ice Hockey**: Ice hockey remains extremely popular in North America and Europe, with the NHL being the most prominent league. The sport is also played at various levels, from amateur to professional, and is a staple of the Winter Olympics.

### Cultural Impact
Hockey, both field and ice, has had a significant cultural impact, particularly in countries where it is a major sport. It has inspired countless players and fans and has become an integral part of national identities, especially in countries like Canada, where ice hockey is considered a national pastime.

Overall, hockey's history is a testament to its enduring appeal and adaptability, evolving from ancient games to a modern sport enjoyed by millions around the world.

**USER:**
Great, but focus specifically on the history of hockey in Canada.

**ASSISTANT:**
Hockey holds a special place in Canadian culture and history, often considered the country's national sport and a key part of its identity. Here's a detailed look at the history of hockey in Canada:

### Early Beginnings
- **Indigenous Influence**: Indigenous peoples in Canada played a variety of stick-and-ball games long before European settlers arrived. These games, such as "shinny," are believed to have influenced the development of ice hockey.
- **European Settlers**: European immigrants brought their own stick-and-ball games to Canada, which evolved into ice hockey. The harsh Canadian winters provided the perfect conditions for playing on frozen ponds and lakes.

### Formalization of the Game
- **First Recorded Game**: The first recorded indoor ice hockey game took place on March 3, 1875, at the Victoria Skating Rink in Montreal. This game is often cited as the beginning of organized ice hockey in Canada.
- **McGill University**: In 1877, students at McGill University in Montreal codified the first set of ice hockey rules, which laid the foundation for the modern game.

### Growth and Popularity
- **Stanley Cup**: In 1893, Lord Stanley of Preston, the Governor General of Canada, donated a trophy to be awarded to the best amateur hockey team in Canada. The Stanley Cup has since become the most prestigious trophy in professional ice hockey.
- **Amateur Leagues**: The late 19th and early 20th centuries saw the formation of numerous amateur leagues across Canada, including the Ontario Hockey Association and the Manitoba Hockey Association.

### Professionalization
- **National Hockey Association (NHA)**: Founded in 1909, the NHA was one of the first professional hockey leagues in Canada. It eventually evolved into the National Hockey League (NHL) in 1917.
- **NHL Expansion**: The NHL began with four Canadian teams and has since expanded to include teams from both Canada and the United States. Canadian teams like the Montreal Canadiens and Toronto Maple Leafs have become iconic franchises.

### International Success
- **Olympic and World Championships**: Canada has a storied history in international hockey, winning numerous Olympic gold medals and World Championships. The country's dominance in the sport has helped solidify its reputation as a hockey powerhouse.
- **Summit Series**: The 1972 Summit Series between Canada and the Soviet Union was a defining moment in Canadian hockey history, showcasing the country's talent and passion for the sport.

### Cultural Impact
- **National Identity**: Hockey is deeply ingrained in Canadian culture, often seen as a symbol of national pride and unity. It is played at all levels, from youth leagues to professional teams, and is a common pastime for many Canadians.
- **Cultural References**: Hockey has been featured in Canadian literature, film, and music, further embedding it in the national consciousness. Iconic moments and players are celebrated and remembered as part of the country's heritage.

### Modern Era
- **Grassroots Development**: Hockey Canada, the governing body for the sport in the country, has focused on developing the game at the grassroots level, ensuring that it remains accessible to all Canadians.
- **Women's Hockey**: Women's hockey has grown significantly in Canada, with the national team achieving great success on the international stage, including multiple Olympic gold medals.

In summary, hockey in Canada is more than just a sport; it is a vital part of the nation's history and cultural fabric. From its early beginnings to its current status as a global leader in the sport, hockey continues to be a source of pride and passion for Canadians.

In [13]:
            input_messages = [
                input_types.ConversationMessageInput(message=message.message, role=message.role)
                for message in messages if message is not None and message.message is not None
            ]

            user_prompt = "Nice, but focus specifically on only the history of Women's Hockey."

            messages = await prompt(user_prompt, specification_id, input_messages)

            if messages is not None:
                display(Markdown('### Conversation:'))

                for message in messages:
                    if message is not None:
                        display(Markdown(f'**{message.role}:**\n{message.message}'))

                        if message.role == enums.ConversationRoleTypes.ASSISTANT:
                            print()


### Conversation:

**USER:**
Discuss the history of hockey.

**ASSISTANT:**
Hockey is a sport with a rich and varied history, evolving over centuries into the modern game we know today. Its origins can be traced back to ancient civilizations, where games involving sticks and balls were played. Here’s a brief overview of the history of hockey:

### Ancient and Medieval Origins
- **Ancient Games**: Games resembling hockey have been played for thousands of years. Evidence suggests that ancient Egyptians, Greeks, and Romans played stick-and-ball games. In particular, a game called "hurling" was played in Ireland, and "shinty" in Scotland, both of which are considered precursors to modern hockey.
- **Medieval Europe**: During the Middle Ages, various stick-and-ball games were played across Europe. These games were often played on fields and involved teams trying to hit a ball into a goal.

### Development of Modern Hockey
- **Field Hockey**: The modern form of field hockey began to take shape in England in the 18th and 19th centuries. The first formal set of rules was established in the mid-19th century, and the first hockey club, Blackheath Hockey Club, was founded in 1861. The sport spread throughout the British Empire, gaining popularity in countries like India and Australia.
- **Ice Hockey**: Ice hockey is believed to have developed in Canada in the 19th century, influenced by various stick-and-ball games brought by European immigrants. The first recorded indoor ice hockey game took place in Montreal in 1875. The sport quickly gained popularity, leading to the formation of amateur clubs and leagues.

### Professionalization and Global Spread
- **National Hockey League (NHL)**: The NHL was founded in 1917 in Montreal, Canada, and has since become the premier professional ice hockey league in the world. It began with just a few teams and has expanded significantly over the years.
- **International Competitions**: Ice hockey became an Olympic sport in 1920, and the International Ice Hockey Federation (IIHF) was established to govern the sport globally. Field hockey was included in the Olympics in 1908 for men and 1980 for women.

### Modern Era
- **Field Hockey**: Today, field hockey is played worldwide, with significant followings in Europe, Asia, Africa, and Oceania. The Hockey World Cup and the Olympic Games are the sport's most prestigious tournaments.
- **Ice Hockey**: Ice hockey remains extremely popular in North America and Europe, with the NHL being the most prominent league. The sport is also played at various levels, from amateur to professional, and is a staple of the Winter Olympics.

### Cultural Impact
Hockey, both field and ice, has had a significant cultural impact, particularly in countries where it is a major sport. It has inspired countless players and fans and has become an integral part of national identities, especially in countries like Canada, where ice hockey is considered a national pastime.

Overall, hockey's history is a testament to its enduring appeal and adaptability, evolving from ancient games to a modern sport enjoyed by millions around the world.

**USER:**
Great, but focus specifically on the history of hockey in Canada.

**ASSISTANT:**
Hockey holds a special place in Canadian culture and history, often considered the country's national sport and a key part of its identity. Here's a detailed look at the history of hockey in Canada:

### Early Beginnings
- **Indigenous Influence**: Indigenous peoples in Canada played a variety of stick-and-ball games long before European settlers arrived. These games, such as "shinny," are believed to have influenced the development of ice hockey.
- **European Settlers**: European immigrants brought their own stick-and-ball games to Canada, which evolved into ice hockey. The harsh Canadian winters provided the perfect conditions for playing on frozen ponds and lakes.

### Formalization of the Game
- **First Recorded Game**: The first recorded indoor ice hockey game took place on March 3, 1875, at the Victoria Skating Rink in Montreal. This game is often cited as the beginning of organized ice hockey in Canada.
- **McGill University**: In 1877, students at McGill University in Montreal codified the first set of ice hockey rules, which laid the foundation for the modern game.

### Growth and Popularity
- **Stanley Cup**: In 1893, Lord Stanley of Preston, the Governor General of Canada, donated a trophy to be awarded to the best amateur hockey team in Canada. The Stanley Cup has since become the most prestigious trophy in professional ice hockey.
- **Amateur Leagues**: The late 19th and early 20th centuries saw the formation of numerous amateur leagues across Canada, including the Ontario Hockey Association and the Manitoba Hockey Association.

### Professionalization
- **National Hockey Association (NHA)**: Founded in 1909, the NHA was one of the first professional hockey leagues in Canada. It eventually evolved into the National Hockey League (NHL) in 1917.
- **NHL Expansion**: The NHL began with four Canadian teams and has since expanded to include teams from both Canada and the United States. Canadian teams like the Montreal Canadiens and Toronto Maple Leafs have become iconic franchises.

### International Success
- **Olympic and World Championships**: Canada has a storied history in international hockey, winning numerous Olympic gold medals and World Championships. The country's dominance in the sport has helped solidify its reputation as a hockey powerhouse.
- **Summit Series**: The 1972 Summit Series between Canada and the Soviet Union was a defining moment in Canadian hockey history, showcasing the country's talent and passion for the sport.

### Cultural Impact
- **National Identity**: Hockey is deeply ingrained in Canadian culture, often seen as a symbol of national pride and unity. It is played at all levels, from youth leagues to professional teams, and is a common pastime for many Canadians.
- **Cultural References**: Hockey has been featured in Canadian literature, film, and music, further embedding it in the national consciousness. Iconic moments and players are celebrated and remembered as part of the country's heritage.

### Modern Era
- **Grassroots Development**: Hockey Canada, the governing body for the sport in the country, has focused on developing the game at the grassroots level, ensuring that it remains accessible to all Canadians.
- **Women's Hockey**: Women's hockey has grown significantly in Canada, with the national team achieving great success on the international stage, including multiple Olympic gold medals.

In summary, hockey in Canada is more than just a sport; it is a vital part of the nation's history and cultural fabric. From its early beginnings to its current status as a global leader in the sport, hockey continues to be a source of pride and passion for Canadians.

**USER:**
Nice, but focus specifically on only the history of Women's Hockey.

**ASSISTANT:**
The history of women's hockey in Canada is a story of perseverance, growth, and success, reflecting broader social changes and the increasing opportunities for women in sports. Here’s an overview of the development of women’s hockey in Canada:

### Early Beginnings
- **19th Century**: The first recorded women's ice hockey game in Canada took place in 1892 in Barrie, Ontario. During this period, women began forming informal teams and playing games, often at local carnivals and winter festivals.
- **Early 20th Century**: By the early 1900s, women's hockey was gaining popularity, with teams forming in cities like Ottawa, Toronto, and Montreal. However, opportunities were limited, and the sport was often seen as a novelty.

### Growth and Challenges
- **1920s and 1930s**: Women's hockey continued to grow, with more teams and leagues forming across Canada. The Preston Rivulettes, one of the most successful teams of the era, dominated the sport, winning numerous championships.
- **World War II**: The war years saw a decline in women's hockey as many players contributed to the war effort, and resources were diverted to the conflict.

### Post-War Resurgence
- **1950s and 1960s**: After World War II, women's hockey experienced a resurgence, with more organized leagues and competitions. However, the sport still faced significant challenges, including limited funding and support.
- **1970s**: The women's liberation movement and increased advocacy for gender equality led to greater opportunities for women in sports, including hockey. This period saw the formation of more teams and leagues, as well as increased media attention.

### Formalization and Recognition
- **1980s**: The Ontario Women's Hockey Association (OWHA) was established in 1975, providing a more structured framework for the sport. The first national women's hockey championship was held in 1982, marking a significant milestone.
- **International Competition**: In 1990, the first IIHF Women's World Championship was held in Ottawa, Canada, with the Canadian team winning the gold medal. This event helped raise the profile of women's hockey on the international stage.

### Olympic Inclusion and Success
- **1998 Nagano Olympics**: Women's ice hockey was included in the Winter Olympics for the first time in 1998. The Canadian team won a silver medal, losing to the United States in the final.
- **2002 Salt Lake City Olympics and Beyond**: Canada won its first Olympic gold medal in women's hockey in 2002, defeating the United States. The Canadian women's team has since become a dominant force, winning multiple Olympic gold medals and World Championships.

### Modern Era and Professionalization
- **Professional Leagues**: The creation of professional leagues, such as the Canadian Women's Hockey League (CWHL) in 2007, provided more opportunities for women to play at a high level. Although the CWHL ceased operations in 2019, efforts continue to establish a sustainable professional league for women.
- **Visibility and Advocacy**: Women's hockey has gained increased visibility, with players advocating for better support, pay, and opportunities. Initiatives like the Professional Women's Hockey Players Association (PWHPA) aim to promote and develop the sport further.

### Cultural Impact
- **Role Models and Inspiration**: Canadian women's hockey players, such as Hayley Wickenheiser, Marie-Philip Poulin, and Manon Rhéaume, have become role models and trailblazers, inspiring future generations of female athletes.
- **Community and Grassroots Development**: Efforts to grow the sport at the grassroots level have led to increased participation among young girls, ensuring a bright future for women's hockey in Canada.

In summary, the history of women's hockey in Canada is marked by significant achievements and ongoing challenges. The sport has grown from its humble beginnings to become a source of national pride, with Canadian women leading the way on the international stage.